# tifftrim: basic demo

This notebook demonstrates the basic functionality of the `tifftrim` package:

1. Install the package in editable mode (recommended when running from the repo)
2. Trim a frame range from a 3D TIFF stack using the Python API
3. (Optional) Verify the output with `tifffile`
4. (Optional) Run the CLI from a notebook cell


In [ ]:
# If you're running this notebook from the repository root, install in editable mode.
# (Re-run if you change the package source.)

!python -m pip install -e .

In [1]:
from pathlib import Path

import tifffile

from tifftrim import trim_3d_tiff

/Users/fpbattaglia/src/TIFFTrim/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set your input TIFF and desired output location.
# The input is expected to load as a 3D array: [frames, y, x].

input_path = Path("/Users/fpbattaglia/Dropbox/Data/477116_20251118_00001.tif")
output_path = input_path.with_name(input_path.stem + "_trimmed.tif")

# Choose frame range (start inclusive, end exclusive)
start_frame = 0
end_frame = 100  # keep frames 0..99

input_path, output_path

(PosixPath('/Users/fpbattaglia/Dropbox/Data/477116_20251118_00001.tif'),
 PosixPath('/Users/fpbattaglia/Dropbox/Data/477116_20251118_00001_trimmed.tif'))

In [4]:
# Trim using the Python API (shows a tqdm progress bar if available)
trim_3d_tiff(
    input_path,
    output_path,
    start_frame,
    end_frame,
    quiet_tifffile_warnings=True,
    show_progress=True,
)

print("Wrote:", output_path)

Wrote: /Users/fpbattaglia/Dropbox/Data/477116_20251118_00001_trimmed.tif


In [5]:
# Quick verification
with tifffile.TiffFile(str(input_path)) as src, tifffile.TiffFile(str(output_path)) as dst:
    src_shape = src.asarray().shape
    dst_shape = dst.asarray().shape

print("Input shape:", src_shape)
print("Output shape:", dst_shape)

Input shape: (40756, 512, 512)
Output shape: (100, 512, 512)


In [ ]:
# Optional: inspect a few tags to confirm the output looks reasonable
with tifffile.TiffFile(str(output_path)) as t:
    page0 = t.pages[0]
    print("First page description present:", page0.description is not None)
    print("First page datetime:", page0.datetime)
    print("First page software (first 200 chars):")
    s = page0.software
    print((s[:200] + "...") if isinstance(s, str) and len(s) > 200 else s)

## Optional: CLI demo

The package also installs a `tifftrim` console script.

Range format: `start:end` where `end` is exclusive (e.g. `0:100`).


In [ ]:
# Run the CLI from within the notebook (uncomment to use)
# !tifftrim -i "/path/to/input.tif" -o "/path/to/output.tif" -r 0:100